In [3]:
import pandas as pd

# Cargar el archivo CSV 'insert_soluciones'
df = pd.read_csv('./sql/insert_empresas.csv')
df.sort_values(by='id')

# Mostrar las primeras filas para verificar la carga
print(df.head(1))


   id           name  sector_id  employees_range  postcode_id  \
0   1  agente prueba        NaN              NaN          NaN   

            updated_at  
0  2023-11-09 14:16:17  


In [ ]:
import psycopg2
import os

DB_NAME = os.getenv('POSTGRES_DB')
USER = os.getenv('POSTGRES_USER')
PASSWORD = os.getenv('POSTGRES_PASSWORD')

# Parámetros de conexión para PostgreSQL (ajusta según tu entorno)
db_params = {
    'host': 'localhost',
    'port': 5432,
    'dbname': DB_NAME,      # Cambia por el nombre real de tu base de datos PostgreSQL
    'user': USER,            # Cambia por tu usuario de PostgreSQL
    'password': PASSWORD      # Cambia por tu contraseña de PostgreSQL
}

# Cargar datos manejando valores NULL correctamente
import numpy as np

print(f"Cargando {len(df)} filas en la tabla empresas_es...")

try:
    # Conectar a la base de datos
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    # Preparar los datos para insertar
    columns = df.columns.tolist()
    
    # 🔧 SOLUCIÓN: Convertir NaN a None para que PostgreSQL los trate como NULL
    df_clean = df.copy()
    df_clean = df_clean.replace({np.nan: None})
    values = df_clean.values.tolist()
    
    print("Valores NaN convertidos a None para PostgreSQL")
    
    # Crear la query de INSERT
    placeholders = ', '.join(['%s'] * len(columns))
    query = f"""
        INSERT INTO empresas_es ({', '.join(columns)})
        VALUES ({placeholders})
    """
    
    # Insertar los datos
    cursor.executemany(query, values)
    
    # Confirmar los cambios
    conn.commit()
    
    print(f"Se insertaron {len(df_clean)} filas correctamente")
    
    # Verificar que se insertaron
    cursor.execute("SELECT COUNT(*) FROM empresas_es")
    count = cursor.fetchone()[0]
    print(f"Total de registros en la tabla: {count}")
    
except Exception as e:
    print(f"Error al insertar datos: {e}")
    if 'conn' in locals():
        conn.rollback()
finally:
    if 'conn' in locals():
        cursor.close()
        conn.close()
